<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2023-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Metodos de Elementos Finitos Mixtos (Mixed Methods) <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Ecuacion de Poisson <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\mbox{div}(c \nabla u) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle c \nabla u\cdot n & = & g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $q$, y $a = c^{-1}$
\begin{equation}
\begin{array}{rclll}
\displaystyle a q & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
\mbox{div}( q ) & = & -f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle q\cdot n & = & g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}

Ahora derivamos la formulacion variacional del sistema de primer order. Primero introducimos espacios apropiados para las funciones test
\begin{equation}
\begin{array}{rcl}
\displaystyle
L_2(\Omega) & = & \{ w : \int_{\Omega} |w|^2 dx < \infty \} \\ 
H(\mbox{div},\Omega) &=& \{ v\in [L_2(\Omega)]^{d}: \mbox{div}(w)\in L_2(\Omega)\}
\end{array}
\end{equation}

Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(q,u) \in H(\mbox{div}, \Omega) \times L_2(\Omega)$ con $q\cdot n = g_{N}$ sobre $\partial \Omega_{N}$ tal que

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} a q \cdot v\,dx + \int_{\Omega} u\,\mbox{div}(v)\,dx & = &
\displaystyle 
\int_{\partial \Omega_{D}} u_{D}v \cdot n ds \\
\displaystyle
\int_{\Omega} \mbox{div}(q)\, w\,dx & = & 
\displaystyle
- \int_{\Omega} f v \,dx
\end{array}
\end{equation}

para $(v,w) \in H(div, \Omega)\times L_2(\Omega)$ con $ v\cdot n = 0$ sobre $\partial \Omega_{N}$.

Notar que, mientras en la formulacion variacional primal la condicion de Neumann aparecia en la formulacion variacional y la condicion de Dirichlet era impueta en el espacio de funciones, en la formulacion mixta es los opuesto.

Definamos la forma bilinear $b:H(\mbox{div}, \Omega)\times L_2(\Omega) \rightarrow \mathbb R$ y el funcional $l:H(\mbox{div}, \Omega)\times L_2(\Omega)$ 

\begin{equation}
\begin{array}{rcl}
\displaystyle b((q,u), (v,w)) &:=& 
\displaystyle \int_{\Omega} a q \cdot v\,dx + \int_{\Omega} u\,\mbox{div}(v)\,dx +\int_{\Omega} \mbox{div}(q)\, w\,dx \\
\displaystyle l(v,w) &:=&
\displaystyle \int_{\partial \Omega_{D}} u_{D}v \cdot n ds - \int_{\Omega} f v \,dx
\end{array}
\end{equation}

Observemos que la forma bilineal $b$ no es coerciva, por lo tanto su analisis y la eleccion de los espacios de elemetnos finitos no tan simple como en el caso de la formulacion primal y el metodo de elementos finitos standard.

[Finite Element Exterior Calculus: from Hodge theory to numerical stability](https://www.ams.org/journals/bull/2010-47-02/S0273-0979-10-01278-4/S0273-0979-10-01278-4.pdf)


In [5]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
#import netgen.gui

# Domain geometry
geo = unit_square
# Generate mesh with meshsize
h = 0.1
mesh = Mesh(geo.GenerateMesh(maxh=h))

# Manufactured solution
uexact = CoefficientFunction( 16*x*(1-x)*y*(1-y) )
qexact = CoefficientFunction((-16*(1-2*x)*y*(1-y), -16*(1-2*y)*x*(1-x)  ))
# Source according 
f      = 32*y*(1-y)+32*x*(1-x)
Draw(uexact, mesh, 'solucion exacta u ')
Draw(qexact, mesh, 'solucion exacta q')
print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

('bottom', 'right', 'top', 'left')


# Elementos finitos Mixtos - Formulacion primal

Ahora definimos el metodos de elemntos finitos para subespacios $X_h\subset L^2(\Omega)^{d}$ y $M_h\subset H_0^1(\Omega)$. Hallar $(q_h,u_h)\in V_h\times W_h$ con $q_h\cdot n = P(g_n)$ sobre $\partial \Omega_N$ tal que

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} a q_h \cdot r\,dx + \int_{\Omega} \nabla u_h\cdot r\,dx & = &
\displaystyle 
0 \\
\displaystyle
\int_{\Omega} q_h\cdot\nabla v\,dx & = & 
\displaystyle
- \int_{\Omega} f v \,dx
\end{array}
\end{equation}
para todo $(r,v)\in X_h\times M_h$

In [6]:
# Mixed finite element spaces
Vh = VectorL2(mesh, order=0)
Wh = H1(mesh, order=1, dirichlet='bottom|right|top|left')

# Mixed space
fesm = FESpace([Vh,Wh])

# variables
q, u = fesm.TrialFunction()
r, v = fesm.TestFunction()

# bilinear form
am = BilinearForm(fesm)
am += (q*r  + q*grad(v) + r*grad(u))*dx
am.Assemble()
fm = LinearForm(fesm)
fm += -f*v*dx 
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")
gfq, gfu = gfm.components


fm.vec.data -= am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs()) * fm.vec
#solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfu, mesh, "u-primal-mixed")
Draw(gfq, mesh, 'q-primal-mixed')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# Elementos finitos Mixtos - formulacion dual

Ahora definimos el metodos de elemntos finitos para subespacios $X_h\subset H(\mbox{div}, \Omega)$ y $M_h\subset L_2(\Omega)$. Hallar $(q_h,u_h)\in X_h\times M_h$ con $q_h\cdot n = P(g_n)$ sobre $\partial \Omega_N$ tal que

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} a q_h \cdot r\,dx - \int_{\Omega} u_h\,\nabla \cdot r\,dx & = &
\displaystyle 
\int_{\partial \Omega_{D}} u_{D}r \cdot n ds \\
\displaystyle
\int_{\Omega} \nabla \cdot q_h v\,dx & = & 
\displaystyle
- \int_{\Omega} f v \,dx
\end{array}
\end{equation}
para todo $(r,v)\in X_h\times M_h$

## Ejemplo de inestabilidad

Escojamos dos subespacios de elementos finitos $ X_h\subset H(\mbox{div}, \Omega)$ y $M_h\in L_2(\Omega)$. Por ejemplo, funciones continuas y lineales a trozos para $X_h$ y funciones discontinuas y constantes a trozos pra $M_h$, es decir
\begin{equation}
\begin{array}{rcl}
X_h &=& \{v\in [C(\Omega)]^{2}: v_i|_K\in \mathcal P_1, \,\forall K \in \mathcal T_h,\, i=1,2\}\\
M_h &=& \{w\in L_2(\Omega): w|_K \in \mathcal P_0, \, \forall K\in \mathcal T_h\}
\end{array}
\end{equation}

Caclumaos la solucion aproximada para el problema con solo condicion de frontera del tipo Dirichlet  $c =1$ y con solucion exacta $u(x,y) = 16 x(1-x)y(1-y)$.

In [7]:
# Mixed finite element spaces
Vh = VectorH1(mesh, order=1)
Wh = L2(mesh, order=0)

# Mixed space
fesm = FESpace([Vh,Wh])
# variables
q, u = fesm.TrialFunction()
r, v = fesm.TestFunction()

# bilinear form
am = BilinearForm(fesm)
am += (q*r - div(q)*v - div(r)*u)*dx
am.Assemble()
fm = LinearForm(fesm)
fm += -f*v*dx 
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")
gfq, gfu = gfm.components


fm.vec.data -= am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs()) * fm.vec
#solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfu, mesh, "u-mixed dual ")
Draw(gfq, mesh, 'q mixed dual')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Espacios de elementos finitos para $H(\mbox{div}, \Omega)$


1. Espacio local de Raviart-Thomas para $K\in \mathcal T_h$

\begin{equation}
\begin{array}{rclrcl}
\mathcal RT_p(K)& =& \mathcal P_p(K)^{d} + x \mathcal P_{p}(K),& \mbox{dim}(\mathcal RT_p(K)) &=& d \binom{p+d}{p} + \binom{p+d-1}{p}
\end{array}
\end{equation}
y espacio $\mathcal P_p(K)$ para la variable escalar.

Espacio global de Raviart-Thomas

\begin{equation}
\mathcal RT_{p}(\mathcal T_h) = \{ v\in H(\mbox{div}, \Omega): v|_{K}\in \mathcal RT_{p}(K) \, \forall K\in \mathcal T_h \}
\end{equation}

#### Propiedad: $\mbox{div}(\mathcal RT_{p}) = \mathcal P_{p}^{d}$

2. Para $d=2$ y $p\geq 1$ y triangulos, definimos el espacio local de Brezzi-Douglas-Marini para $K\in \mathcal T_h$

\begin{equation}
\begin{array}{rclrcl}
\mathcal BDM_p(K)& =& \mathcal P_p(K)^{d},& \mbox{dim}(\mathcal BDM_p(K)) &=& (p+1)(p+2)
\end{array}
\end{equation}
y espacio $\mathcal P_{p-1}(K)$ para la variable escalar.

Espacio global de Brezzi-Douglas-Marini

\begin{equation}
\mathcal BDM_{p}(\mathcal T_h) = \{ v\in H(\mbox{div}, \Omega): v|_{K}\in \mathcal BDM_{p}(K) \, \forall K\in \mathcal T_h \}
\end{equation}



In [8]:
# Mixed finite elements spaces
Vh = HDiv(mesh,order=0, RT=True) # Raviart-Thomas space
#Vh = HDiv(mesh,order=0) # Brezzi-Douglas-Marini space
Wh = L2(mesh, order=0)
fesm = FESpace([Vh,Wh])
# 
q, u = fesm.TrialFunction()
r, v = fesm.TestFunction()

# bilinear form
am = BilinearForm(fesm)
am += (q*r - div(q)*v - div(r)*u)*dx
am.Assemble()
fm = LinearForm(fesm)
fm += -f*v*dx 
fm.Assemble()

gfm = GridFunction(fesm, name="gfmixed")
gfq, gfu = gfm.components

fm.vec.data -= am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs()) * fm.vec
#solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfu, mesh, "u-mixed-dual")
Draw (gfq, mesh, 'q-mixed-dual')
#Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene